In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-03"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1937,2024-06-03,Turquia Superliga,14:30,Besiktas,Fenerbahce,2.99,1.37,163.5,1.82,1.92,0.0,0.00,0.00,d47V2UsT,0.334448,0.729927,0.549451,0.520833,0.064375,204.894,120.405975,0.587650,1.8,1.643168,0.912871,285.84,2.426,1.572714,0.648275,49.0,112.780,7.079294,0.062771,3.0,0.000000,0.000000,120.54,1.156,0.067305,0.058223,95.0,72,98,3.97,1.23,120.198,123.258,0.525465,0.037813,NaN,0.99,0.198,10.050505,0.783422,1.0,0.216578,0.10,0.020,18.500000,0.855181,0.9,0.044819
1938,2024-06-03,Porto Rico Bsn,21:00,Criollos de Caguas,Cangrejeros,1.66,2.13,184.5,1.83,1.85,0.0,0.00,0.00,UqODMOjp,0.602410,0.469484,0.546448,0.540541,0.071893,184.270,59.798229,0.324514,1.2,1.643168,1.369306,274.17,1.958,0.376855,0.192470,-34.0,148.708,30.592533,0.205722,0.6,1.341641,2.236068,130.20,1.706,0.307295,0.180126,-15.0,111,84,2.47,1.55,145.098,221.210,0.175377,0.007686,NaN,0.82,0.164,4.024390,0.000000,0.0,0.000000,-2.44,-0.488,-2.315574,0.000000,0.0,0.000000
1939,2024-06-03,Colômbia Lbp,19:00,Titanes de Barranquilla,Caribbean Storm Islands,1.10,6.15,153.5,1.86,1.84,0.0,0.00,0.00,nZFwOXmB,0.909091,0.162602,0.537634,0.543478,0.071693,117.690,33.169736,0.281840,1.8,1.643168,0.912871,108.78,1.578,0.458007,0.290245,6.0,136.268,42.830007,0.314307,3.0,0.000000,0.000000,197.12,1.616,0.616060,0.381225,84.0,74,77,1.47,2.56,97.224,0.000,0.985073,0.007644,NaN,-0.40,-0.080,-1.250000,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1940,2024-06-03,Brasil Lbf Feminina,19:30,Sampaio F,Ituano F,1.07,7.11,142.5,1.83,1.87,0.0,0.00,0.00,lGij0B0T,0.934579,0.140647,0.546448,0.534759,0.075226,491.060,844.112310,1.718960,3.0,0.000000,0.000000,2000.00,5.080,8.353125,1.644316,140.0,86.288,12.387622,0.143561,2.4,1.341641,0.559017,79.80,1.148,0.123774,0.107817,3.0,100,76,20.00,1.05,92.212,86.188,1.044236,0.015289,NaN,0.85,0.170,0.411765,0.000000,0.0,0.000000,-0.32,-0.064,-95.468750,0.000000,0.0,0.000000
1941,2024-06-03,Brasil Lbf Feminina,19:30,Corinthians F,Blumenau F,2.24,1.60,132.5,1.82,1.88,-2.5,1.87,1.83,YwQ8IhGc,0.446429,0.625000,0.549451,0.531915,0.071429,136.602,49.176202,0.359996,1.2,1.643168,1.369306,84.53,2.138,0.887085,0.414914,-17.0,226.226,230.936340,1.020821,1.2,1.643168,1.369306,619.50,3.682,3.920296,1.064719,-36.0,79,59,1.07,10.50,136.688,357.124,0.235702,0.022933,0.015289,-1.24,-0.248,-5.000000,0.000000,0.0,0.000000,0.52,0.104,5.769231,0.000000,0.0,0.000000
1942,2024-06-03,Brasil Lbf Feminina,18:30,Santo André F,Catanduva F,1.02,11.60,141.5,1.83,1.83,0.0,0.00,0.00,S03s2XVG,0.980392,0.086207,0.546448,0.546448,0.066599,164.798,121.261287,0.735818,2.4,1.341641,0.559017,59.28,2.104,1.190643,0.565895,77.0,499.358,295.635238,0.592031,0.6,1.341641,2.236068,557.55,7.738,4.225023,0.546010,-69.0,57,59,1.04,9.45,241.630,425.768,1.185609,0.000000,NaN,7.56,1.512,0.013228,0.000000,0.0,0.000000,-5.00,-1.000,-10.600000,0.000000,0.0,0.000000
1943,2024-06-03,Sérvia Superliga,15:00,Crvena zvezda,Partizan,1.40,2.80,165.5,1.87,1.79,0.0,0.00,0.00,l81EbpNF,0.714286,0.357143,0.534759,0.558659,0.071429,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,98,96,1.20,1.23,0.000,0.000,0.471405,0.030912,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.0000

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
1941,19:30,Brasil Lbf Feminina,Corinthians F,Blumenau F,2.24,Back Home
1944,21:00,Porto Rico Bsn,Osos de Manati,Gigantes de Carolina,1.65,Back Home
1947,21:30,Bolívia Libobasquet,Kinwa,Can Oruro,2.12,Back Home
